In [1]:
!pip install -q huggingface_hub boto3 


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os
import boto3
from huggingface_hub import hf_hub_download
from huggingface_hub import snapshot_download

In [3]:
!pwd
!mkdir -p /opt/app-root/src/llm

/opt/app-root/src


In [6]:
bucket_name = "llm"
base_path = "/opt/app-root/src/llm"
hf_token = "your huggingface token"

s3_client = boto3.client('s3',
                         aws_access_key_id='your minio access key',
                         aws_secret_access_key='your minio secret key',
                         endpoint_url="your minio endpoint"
                        )


def upload_file_to_minio(repo_id, prefix_key, fullfilename=None):
    path = f"{base_path}/{prefix_key}" if fullfilename is None else fullfilename
    for filename in os.listdir(path):
        fullfilename = path + '/' + filename
        print("Uploading:", fullfilename)
        if os.path.isdir(fullfilename):
            upload_file_to_minio(repo_id, prefix_key, fullfilename)
        else:
            try:
                s3_client.upload_file(fullfilename, bucket_name, prefix_key + '/' + filename)
            except Exception as e:
                print(f"An error occurred while uploading {fullfilename}: {e}")

snapshot_download(repo_id='meta-llama/Llama-3.1-8B-Instruct', cache_dir=f"{base_path}/llama", token=hf_token)
upload_file_to_minio('meta-llama/Llama-3.1-8B-Instruct', "llama")

snapshot_download(repo_id='ibm-granite/granite-3.0-8b-instruct', cache_dir=f"{base_path}/granite", token=hf_token)
upload_file_to_minio('ibm-granite/granite-3.0-8b-instruct', "granite")

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/3.48M [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.41G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/5.64k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

Uploading: /opt/app-root/src/llm/granite/models--ibm-granite--granite-3.0-8b-instruct
Uploading: /opt/app-root/src/llm/granite/models--ibm-granite--granite-3.0-8b-instruct/refs
Uploading: /opt/app-root/src/llm/granite/models--ibm-granite--granite-3.0-8b-instruct/refs/main
Uploading: /opt/app-root/src/llm/granite/models--ibm-granite--granite-3.0-8b-instruct/blobs
Uploading: /opt/app-root/src/llm/granite/models--ibm-granite--granite-3.0-8b-instruct/blobs/0a11f2016e660fd490f7bf168e6d1f9c86a8f744
Uploading: /opt/app-root/src/llm/granite/models--ibm-granite--granite-3.0-8b-instruct/blobs/aa755f8f4af8b1eb999e60800ab12661442a6ef5
Uploading: /opt/app-root/src/llm/granite/models--ibm-granite--granite-3.0-8b-instruct/blobs/d74ee73a87b784306471df88cdd53b819b0d3450
Uploading: /opt/app-root/src/llm/granite/models--ibm-granite--granite-3.0-8b-instruct/blobs/11f3f2c66235d30bc9bf04ad7e1b4a19d80e84da
Uploading: /opt/app-root/src/llm/granite/models--ibm-granite--granite-3.0-8b-instruct/blobs/79c434f5c43

In [7]:
!rm -rf /opt/app-root/src/llm/